# Function calling

Function calling is a powerful feature of OpenAI `gpt-4` and other models. It essentially is a **Function Picker and Parameter Provider**. It can help you choose which function, if any, to invoke with which parameters, under provided context and instruction. 

LionAGI allows simple usage of function callings in the `Session` object. 

In [11]:
import lionagi as li

In [12]:
# let us define a function description in OpenAI schema

tools = [
    {
        "type": "function",
        "function": {
            "name": "multiply",
            "description": "Perform multiplication on two numbers",
            "parameters": {
                "type": "object",
                "properties": {
                    "number1": {
                        "type": "number",
                        "description": "a number to multiply, e.g. 5.34",
                    },
                    "format": {
                        "number2": "number",
                        "description": "a number to multiply, e.g. 17",
                    },
                },
                # specify which parameters are required for the model to respond when function calling
                "required": ["number1", "number2"],
            },
        }
    }
]

In [ ]:
# and now some messages
system = "It is now May, take a deep breath and think step by step, Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."
task = "pick two numbers as parameters for function calling from the question. The purpose is to show basic principle of multiplication"

# when using respond_mode as json to enforce output format, you need to provide specifying details in instruction
json_format = {"number1": "x", "number2": "y"}

instruct1 = {"Task": task, "json_format": json_format}

In [13]:
question = "There are [basketball, baseball, football, bags, water bottles, strawberry, tennis ball, ping pong table]. each comes in four different colors, how many unique kinds of balls are there?"


context1 = {"Question": question}

In [14]:
# iniate the session
calculator = li.Session(system)

# update the default llmconfig to include the tools if you wish to make the tools accessible for every message within the whole session
calculator.llmconfig.update({
    "tools": tools,
    "tool_choice": "auto",
    "response_format": {'type':'json_object'}
})

In [15]:
calculator.llmconfig

{'model': 'gpt-4-1106-preview',
 'frequency_penalty': 0,
 'max_tokens': None,
 'n': 1,
 'presence_penalty': 0,
 'response_format': {'type': 'json_object'},
 'seed': None,
 'stop': None,
 'stream': False,
 'temperature': 0.7,
 'top_p': 1,
 'tools': [{'type': 'function',
   'function': {'name': 'multiply',
    'description': 'Perform multiplication on two numbers',
    'parameters': {'type': 'object',
     'properties': {'number1': {'type': 'number',
       'description': 'a number to multiply, e.g. 5.34'},
      'format': {'number2': 'number',
       'description': 'a number to multiply, e.g. 17.0'}},
     'required': ['number1', 'number2']}}}],
 'tool_choice': 'auto',
 'user': None}

In [16]:
out = await calculator.initiate(instruction=instruct1,context=context1)
print(out)


    {"tool_uses": [
      {
        "recipient_name": "functions.multiply",
        "parameters": {
          "number1": 4,
          "number2": 4
        }
      }
    ]}
  


In [17]:
import json

out1 = json.loads(out)
func = out1['function'][5:]
arguments = json.loads(out1['arguments'])

KeyError: 'function'

In [10]:
func

'multiply'

In [8]:
out1

{'function': 'func_multiply', 'arguments': '{"number1":4,"number2":3}'}

In [ ]:
out

In [21]:
arguments.keys()

dict_keys(['number1', 'number2'])

{'number1': 4, 'number2': 3}

In [13]:
out

'{"function": "func_multiply", "arguments": "{\\"number1\\":4,\\"number2\\":3}"}'

In [6]:
calculator.conversation.messages[-1]

{'role': 'assistant',
 'content': '{"function": "func_multiply", "arguments": "{\\"number1\\":12,\\"number2\\":4}"}'}

In [ ]:
calculator._logger.log

deque([{'input': {'messages': [{'role': 'system',
           'content': "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."},
          {'role': 'user',
           'content': '{"instruction": {"instruction": "provide two numbers for multiplication for the following question. The purpose is to show basic principle of multiplication: There are basketball, baseball and football, each comes in four different colors, how many unique kinds of balls are there?", "json_format": {"number1": "xx", "number2": "yy"}}}'},
          {'role': 'assistant',
           'content': '{"function": "func_multiply", "arguments": "{\\"number1\\":12,\\"number2\\":4}"}'}],
         'model': 'gpt-4-1106-preview',
         'frequency_penalty': 0,
         'n': 1,
         'presence_penalty': 0,
         'response_format': {'type': 'json_object'},
         'temperature': 0.7,
         'top_p': 1,
         'tools': [{'type': 'function',
           

In [8]:
out

'{"function": "func_multiply", "arguments": "{\\"number1\\":12,\\"number2\\":4}"}'

In [9]:
bool("none")

True